Before start

We use Gemini 1.5 Flash (simmilar to Claude3 Haiku, but compare its Cost effectiveness : Gemini 1.5 Flash is much better in 2024. may 16)

`pip install -q -U google-generativeai`

Working well. now use it to owr usecase.

In [5]:
## Before start, load parsed document from my parsing method: table handling with tatr
from pathlib import  Path
from document_preprocess import process_pdf_text_from_plumber, postprocess_with_datr

# Example usage
base_path = Path("preprocess_document/output")
pdf_path = base_path / "this_is_sample.pdf"

detection_path = base_path / "results/detection"
structure_path = base_path / "results/structure"

# base_path의 마지막 디렉토리 이름과 pdf_path의 파일 이름(확장자를 제외한 부분)을 결합하여 file_prefix 생성
file_prefix = f"{pdf_path.stem}"

pdfplumber_extracted_text = process_pdf_text_from_plumber(pdf_path, detection_path, file_prefix)

# for element in pdfplumber_extracted_text:
#     print(f"Type: {element.type}, Page: {element.page_number}, Table Index: {element.table_index}")
#     print(element.text)
#     print("-" * 80)

postprocess_result = postprocess_with_datr(pdfplumber_extracted_text, structure_path, file_prefix)

for element in postprocess_result:
    print(f"Type: {element.type}, Page: {element.page_number}, Table Index: {element.table_index}")
    print(element.text)
    print(element.bbox)
    print("-" * 80)

Type: table, Page: 1, Table Index: 1
| MATERIAL Case Carburising Steel 8319M (20MnCrNiMo5) | Standard Number: |
| :--- | :--- |
|
|
|

(97.919960832, 104.05589197775328, 141.71166609625004, 113.4182882327932)
--------------------------------------------------------------------------------
Type: text, Page: 1, Table Index: None
Contents: Purpose Field of Validity Process Information Chemical Composition Grain Size Cleanliness Hardenability Delivery Standards Related Standards Purpose: The purpose of this Standard is to define requirements for the steel grade 8319M. Field of Validity: This Standard defines the steel to be used for the manufacture of carburised or carbonitrided Driveline components, e.g. gears. Process Information: This steel can undergo a combination of the following processes: · cold-forming, warm or hot forging · soft-machining · case carburising or carbonitriding · induction or furnace hardening · tempering · hard-machining · welding (laser, electron beam or MAG) · ma

In [6]:
pages = {}
for element in postprocess_result:
    if element.page_number not in pages:
        pages[element.page_number] = []
    pages[element.page_number].append(element)

for page_number, elements in pages.items():
    print(f"Page {page_number}")
    for element in elements:
        print(f"Type: {element.type}, Table Index: {element.table_index}")
        print(element.text)
        print("-" * 80)
    print("=" * 100)

Page 1
Type: table, Table Index: 1
| MATERIAL Case Carburising Steel 8319M (20MnCrNiMo5) | Standard Number: |
| :--- | :--- |
|
|
|

--------------------------------------------------------------------------------
Type: text, Table Index: None
Contents: Purpose Field of Validity Process Information Chemical Composition Grain Size Cleanliness Hardenability Delivery Standards Related Standards Purpose: The purpose of this Standard is to define requirements for the steel grade 8319M. Field of Validity: This Standard defines the steel to be used for the manufacture of carburised or carbonitrided Driveline components, e.g. gears. Process Information: This steel can undergo a combination of the following processes: · cold-forming, warm or hot forging · soft-machining · case carburising or carbonitriding · induction or furnace hardening · tempering · hard-machining · welding (laser, electron beam or MAG) · manganese phosphating Chemical Composition: 
------------------------------------------

In [7]:
import pymupdf, pymupdf4llm
input_pdf = "preprocess_document/output/this_is_sample.pdf"

md_text = pymupdf4llm.to_markdown(str(pdf_path), page_chunks=True)

In [8]:
for page in md_text:
    print("this is page", page['metadata']['page'])
    print(page['text'])

this is page 1
|C [%]|Si [%]|Mn [%]|P [%]|S [%]|Cr [%]|Mo [%]|Ni [%]|Cu [%]|Al [%]|N [%]|Al/N|
|---|---|---|---|---|---|---|---|---|---|---|---|
|0.18 0.23|0.15 0.35|1.10 1.45|max. 0.025|0.020 0.035|0.70 0.95|0.25 0.35|0.25 0.40|max. 0.30|0.020 0.060|Max. 0.015|2 min.|

|Ti [ppm]|B [ppm]|O [ppm]|Cu+10×Sn [%]|Nb [%]|Sb [%]|
|---|---|---|---|---|---|
|max. 40|max. 5|max. 25|max. 0.50|max. 0.010|max. 0.010|


![this_is_sample.pdf-0-1.png](this_is_sample.pdf-0-1.png)

![this_is_sample.pdf-0-0.png](this_is_sample.pdf-0-0.png)

000175B1_DOE/25.feb.2003

|Col1|Col2|MATERIAL Case Carburising Steel 8319M (20MnCrNiMo5)|Col4|Standard Number:|Col6|Col7|Col8|
|---|---|---|---|---|---|---|---|
|||||502118||||
|||||Page|of||Issue Level|
|||||1|2||A|
|Contents: Purpose Field of Validity Process Information Chemical Composition Grain Size Cleanliness Hardenability Delivery Standards Related Standards Purpose: The purpose of this Standard is to define requirements for the steel grade 8319M. Field of Val

In [9]:
md_text = pymupdf4llm.to_markdown(str(pdf_path))

print(md_text)

|C [%]|Si [%]|Mn [%]|P [%]|S [%]|Cr [%]|Mo [%]|Ni [%]|Cu [%]|Al [%]|N [%]|Al/N|
|---|---|---|---|---|---|---|---|---|---|---|---|
|0.18 0.23|0.15 0.35|1.10 1.45|max. 0.025|0.020 0.035|0.70 0.95|0.25 0.35|0.25 0.40|max. 0.30|0.020 0.060|Max. 0.015|2 min.|

|Ti [ppm]|B [ppm]|O [ppm]|Cu+10×Sn [%]|Nb [%]|Sb [%]|
|---|---|---|---|---|---|
|max. 40|max. 5|max. 25|max. 0.50|max. 0.010|max. 0.010|


![this_is_sample.pdf-0-1.png](this_is_sample.pdf-0-1.png)

![this_is_sample.pdf-0-0.png](this_is_sample.pdf-0-0.png)

000175B1_DOE/25.feb.2003

|Col1|Col2|MATERIAL Case Carburising Steel 8319M (20MnCrNiMo5)|Col4|Standard Number:|Col6|Col7|Col8|
|---|---|---|---|---|---|---|---|
|||||502118||||
|||||Page|of||Issue Level|
|||||1|2||A|
|Contents: Purpose Field of Validity Process Information Chemical Composition Grain Size Cleanliness Hardenability Delivery Standards Related Standards Purpose: The purpose of this Standard is to define requirements for the steel grade 8319M. Field of Validity: This Sta

In [10]:
import google.generativeai as genai

model = genai.GenerativeModel('gemini-1.5-flash')